specific humidity plot process:
- only read in 11:30am files for warm season
- interpolate dataset
- calculate q for every 20m height level from surface to 4km and for near surface average layer (0 to 1km)
- find correlation coefficients between near-surface & all other levels

questions:
- linear regression part -- what are we doing here
- above 2km still "significantly correlated" is it? decreases to about 0.3 which doesn't seem significant

next step: 
issues in raw data, need to handle messy data by either filling the invalid data with NANs or dropping the files entirely (one issue is that theres a pressure value of 19 at the beginning of the file but its not invalid)

have to calculate mixed layer specific humidity by either taking average of specific humidity in the lowest 1km or mixed_parcel dewpoint --> specific humidity

figure out how to do standard error on correlation coefficients


In [50]:
import pandas as pd
import glob
import os
import re
import datetime
import metpy.calc as mpcalc
from metpy.units import units
from metpy.cbook import get_test_data
from metpy.plots import add_metpy_logo, SkewT
from pyhdf import SD
from scipy.interpolate import interp1d
from scipy.optimize import bisect
import numpy as np
import pickle
import numpy as np
import math
import pint
from pydoc import help
from pyhdf.SD import *
import xarray as xr
import netCDF4 as nc
import re
import matplotlib.pyplot as plt
from numpy import load
from numpy import asarray
from numpy import save
import pytz
from scipy import stats
from sklearn.linear_model import LinearRegression
from collections import Counter
import pymannkendall as mk
%matplotlib inline

In [87]:
# Set display options to show all rows and columns in dataframe
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

-- interpolation functions --

In [281]:
def interpolate_height(row):
    x = file.variables["alt"].data
    max_x = 12000.0
    xnew = np.arange(min(x),max_x,20)
    xnew = xnew * units.m
    
    plt.plot(x, y, 'b-', label='Original Data')
    plt.scatter(xnew, ynew, c='red', marker='o', s=10, label='Interpolated Point')
        
        # Customize the plot
    plt.xlabel('Altitude')
    plt.ylabel('Dewpoint')
    plt.legend()
        
    # Show the plot
    plt.show()
    return(xnew)

def interpolate_temperature(file):
    x = file.variables["alt"].data
    y = file.variables["tdry"].data
    
    # Interpolation object
    f = interp1d(x, y, assume_sorted = False)
    max_x = 12000.0
    xnew = np.arange(min(x),max_x,20)
    ynew = f(xnew)*units.degC
    return(ynew)

def interpolate_dewpoint(file):
    x = file.variables["alt"].data
    y = file.variables["dp"].data
    
    # Interpolation object
    f = interp1d(x, y, assume_sorted = False)
    max_x = 12000.0
    xnew = np.arange(min(x),max_x,20)
    ynew = f(xnew)*units.degC
    
# #     plt.plot(x, y, 'b-', label='Original Data')
# #     plt.scatter(xnew, ynew, c='red', marker='o', s=10, label='Interpolated Point')
        
# #         # Customize the plot
# #     plt.xlabel('Altitude')
# #     plt.ylabel('Dewpoint')
# #     plt.legend()
        
# #     # Show the plot
# #     plt.show()
#     return(ynew)

def interpolate_pressure(file):
    x = file.variables["alt"].data
    y = file.variables["pres"].data
    
    # Interpolation object
    f = interp1d(x, y, assume_sorted = False)
    max_x = 12000.0
    xnew = np.arange(min(x),max_x,20)
    ynew = f(xnew)*units.hPa
    return(ynew)


In [66]:
def time_adjustment(file):
    bt = file.variables["base_time"].data
    t = file.coords["time"].data[0]
    t_off = file.variables["time_offset"].data[0]
    
    bt = pd.Timestamp(bt)
    t_off = pd.Timestamp(t_off)
    
    yr = bt.year
    month = bt.month
    day = bt.day
    
    if bt.hour == 0 and bt.minute == 0 and bt.second == 0:
        hr = t_off.hour
        minute = t_off.minute
        second = t_off.second
    else:
        hr = bt.hour
        minute = bt.minute
        second = bt.second
    
    time_final = (yr, month, day, hr, minute, second)
    time_final = datetime.datetime(*time_final)
    #darmdata_utc_times.append(time_final)
    return(time_final)

In [67]:
# Define a function to map month numbers to seasons
def assign_season(month):
    if month in [3,4,5]:
        return 'MMA'
    elif month in [6,7,8]:
        return 'JJA'
    elif month in [9,10,11]:
        return 'SON'
    else:
        return 'DJF'

In [272]:
dps = []
dp_errors=[]
for file in darmdata_cleaned:
    try:
        time = time_adjustment(file)
        dp = file.variables["dp"].data
        h = file.variables["alt"].data
        h_interp = interpolate_height(file)
        dp_interp = interpolate_dewpoint(file)
        tup = (time,h,h_interp,dp,dp_interp)
        dps.append(tup)
    except Exception as e:
        dp_errors.append(dp)
        #print(e)

dpdf = pd.DataFrame(dps,columns=['time','h','h_interp','dp','interpdp'])

/data/rong3/annie/anaconda3/envs/annieenv/lib/python3.9/site-packages/scipy/interpolate/interpolate.py:630: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/data/rong3/annie/anaconda3/envs/annieenv/lib/python3.9/site-packages/scipy/interpolate/interpolate.py:633: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo
/data/rong3/annie/anaconda3/envs/annieenv/lib/python3.9/site-packages/scipy/interpolate/interpolate.py:630: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]


In [273]:
dpdf = dpdf.set_index("time")
dpdf.index = pd.to_datetime(dpdf.index)
dpdf.index = dpdf.index - datetime.timedelta(hours=6)

In [275]:
dt = pd.to_datetime('2008-06-13 11:31:00')

# Use .loc to select the row by the datetime index and access the 'dp' column
dp_error_ex = dpdf.loc[dt,:]

dp_error_ex


h           [19.0, 315.0, 321.0, 330.0, 336.0, 345.0, 351....
h_interp    [19.0 meter, 39.0 meter, 59.0 meter, 79.0 mete...
dp          [17.7, 17.4, 17.3, 17.0, 16.9, 16.7, 16.7, 16....
interpdp    [17.700000762939453 degree_Celsius, 17.6797304...
Name: 2008-06-13 11:31:00, dtype: object

In [289]:
darmdata_cleaned[1]

AttributeError: 'Frozen' object has no attribute 'data'

In [256]:
specific_row = sh_df.loc[specific_date]
for i in specific_row['height']:
    print(i)

19.0 meter
39.0 meter
59.0 meter
79.0 meter
99.0 meter
119.0 meter
139.0 meter
159.0 meter
179.0 meter
199.0 meter
219.0 meter
239.0 meter
259.0 meter
279.0 meter
299.0 meter
319.0 meter
339.0 meter
359.0 meter
379.0 meter
399.0 meter
419.0 meter
439.0 meter
459.0 meter
479.0 meter
499.0 meter
519.0 meter
539.0 meter
559.0 meter
579.0 meter
599.0 meter
619.0 meter
639.0 meter
659.0 meter
679.0 meter
699.0 meter
719.0 meter
739.0 meter
759.0 meter
779.0 meter
799.0 meter
819.0 meter
839.0 meter
859.0 meter
879.0 meter
899.0 meter
919.0 meter
939.0 meter
959.0 meter
979.0 meter
999.0 meter
1019.0 meter
1039.0 meter
1059.0 meter
1079.0 meter
1099.0 meter
1119.0 meter
1139.0 meter
1159.0 meter
1179.0 meter
1199.0 meter
1219.0 meter
1239.0 meter
1259.0 meter
1279.0 meter
1299.0 meter
1319.0 meter
1339.0 meter
1359.0 meter
1379.0 meter
1399.0 meter
1419.0 meter
1439.0 meter
1459.0 meter
1479.0 meter
1499.0 meter
1519.0 meter
1539.0 meter
1559.0 meter
1579.0 meter
1599.0 meter
1619.0 meter
16

In [62]:
#loop to read in all sondewnpn files
darmdata = []
path = "../../data/rong1/climate_fu/Data/DOEARM/sgp/sondewnpn"
file_list = sorted(glob.glob(path + "/*.cdf"))
for file in file_list: 
    ds = xr.open_dataset(file)
    darmdata.append(ds)
    
# #loop to remove files that have less than a 50hpa range in pres variable
# darmdata_cleaned = []
# count_fails = 0
# for file in darmdata: 
#     pres = file.variables["pres"].data
#     first_p = pres[0]
#     last_p = pres[-1]
#     if first_p - last_p >= 50:
#         darmdata_cleaned.append(file)
#     else:
#         count_fails += 1

In [ ]:
def clean_xarray_datasets(datasets):
    cleaned_datasets = []

    for ds in darmdata:
        # Create a copy of the dataset to avoid modifying the original
        cleaned_ds = ds.copy()

        # List of variables to check and replace invalid values with NaN
        variables_to_check = ['pres', 'alt', 'dp','temp']  # Replace with your variable names
        
        for variable_name in variables_to_check:
            # Check for invalid values in the variable (e.g., values greater than a threshold)
            invalid_values = cleaned_ds[variable_name] > invalid_threshold

            # Replace invalid values with NaN
            cleaned_ds[variable_name] = cleaned_ds[variable_name].where(~invalid_values, np.nan)

        # Append the cleaned dataset to the list
        cleaned_datasets.append(cleaned_ds)

    return cleaned_datasets

In [280]:
darmdata_cleaned[0].variables['pres']

<xarray.Variable (time: 3081)>
array([973.3, 971.8, 970.9, ...,  47.9,  47.9,  47.8], dtype=float32)
Attributes:
    long_name:    Pressure
    units:        hPa
    resolution:   0.1
    field_index:  1
    calc:         pres 
    min:          0.0
    max:          1100.0
    delta:        10.0

In [2]:
#combine all files into one for warm season @ 11:30am and only pressure, dewpoint, temperature from each file

In [72]:
sh_list = []
errors=0
for file in darmdata_cleaned:
    try:
        time = time_adjustment(file)
        pres = interpolate_pressure(file)
        temp = interpolate_temperature(file)
        height = interpolate_height(file)
        dp = interpolate_dewpoint(file)
        tup = (time,pres,temp,height,dp)
        print(len(pres),len(temp),len(height),len(dp))
        sh_list.append(tup)
    except Exception as e:
        errors+=1
        print(e)

sh_df = pd.DataFrame(sh_list,columns=['time','pres','temp','height','dp'])

585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
A value in x_new is above the interpolation range.
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 5

/data/rong3/annie/anaconda3/envs/annieenv/lib/python3.9/site-packages/scipy/interpolate/interpolate.py:630: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]


585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
A value in x_new is above the interpolation range.
585 585 585 585
585 585 585 585
A value in x_new is above the interpolation range.
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
A value in x_new is above the interpolation range.
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585

585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
A value in x_new is above the interpolation range.
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
A value in x_new is above the interpolation range.
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
A value in x_new is above the interpolation range.
585 585 585 585
585 585 585 585
A value in x_new is above the interpolation range.
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 

585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
A value in x_new is above the interpolation range.
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
A value in x_new is above the interpolation range.
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
58

...

585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 

/data/rong3/annie/anaconda3/envs/annieenv/lib/python3.9/site-packages/scipy/interpolate/interpolate.py:630: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]


585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
A value in x_new is above the interpolation range.
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
A value in x_new is above the interpolation range.
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
A value in x_new is above the interpolation range.
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
A value in x_new is above the interpolation range.
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 

585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 

585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 585 585
585 585 

In [75]:
#filtering sh_df to only warm season & 11:30CDT
sh_df = pd.DataFrame(sh_list,columns=['time','pres','temp','height','dp'])
sh_df = sh_df.set_index("time")
sh_df.index = pd.to_datetime(sh_df.index)
sh_df.index = sh_df.index - datetime.timedelta(hours=6)
sh_df['year'] = sh_df.index.year
sh_df['month'] = sh_df.index.month
sh_df['day'] = sh_df.index.day
sh_df['hour'] = sh_df.index.hour
sh_df['season'] = sh_df['month'].apply(assign_season)

months_to_filter = [5, 6, 7, 8, 9] 
# Filter sh_df for rows with hour equal to 11 and month in the specified list of months
sh_df = sh_df[(sh_df['hour'] == 11) & (sh_df['month'].isin(months_to_filter))]

In [80]:
#spec_hum_calc calculates specific humidity from pressure and dewpoint at every 20m height level
#creates a specific humidity array & appends to the spechum column
def spec_hum_calc(row):
    height_array = row['height'][:200]
    pressure_array = row['pres'][:200]
    dewpoint_array = row['dp'][:200]
    #length = len(height_array)
    
    sh_list=[]
    
    #only calculating specific humidity up to 4km
    for i in range(len(height_array)):
        sh = mpcalc.specific_humidity_from_dewpoint(pressure_array[i],dewpoint_array[i]).to('g/kg')
        sh_list.append(sh.magnitude)
    print(min(sh_list),max(sh_list))
    #print(sh_list)
    #plt.plot(sh_list,height_array)
    #plt.show()
    return sh_list


sh_df['spechum'] = sh_df.apply(spec_hum_calc, axis=1)

2.715747670759052 13.404338165551119
3.96604170232629 13.05799786096342
1.0623100485882133 11.225772034742164
2.6018077299621134 10.969467586259482
0.7532187407847524 12.303832487663191
0.8308034186433491 13.558570037250078
0.6108782332313011 13.078092210536514
5.245548010170219 12.715632508746225
6.043264160501425 10.404186130540626
5.972611174209164 14.668283542158129
1.387207182477417 15.221251889132311
3.739297345164055 12.079113029890335
2.131795619095972 11.675478330713624
2.9596503627914608 11.60181155570758
4.558144560325153 13.496423262826553
0.23997357998052107 15.472716951328573
1.5024493466240676 10.247829225602732
2.4603400585080006 10.998876264998467
1.0017204783202518 13.414561719306725
4.922921462702276 14.298809285834784
4.632560472879274 14.549784596419306
4.775788157952111 12.35405003475873
3.529577147454894 12.36485481205066
2.0036256099680303 15.054002007595843
0.7333650991651761 12.26392719790735
3.40756521446205 13.64483497007347
3.8252987135869856 12.95858243083

0.7303561925920494 6.252740932401332
0.48850653098633867 7.3941224890307815
2.0572851227390356 8.840593316115061
3.9675968788226754 12.926273609727854
0.12122034048878678 8.585908690602283
0.07202888651326878 7.95851220953063
0.07430044427409356 7.16286807313127
3.769224002641999 10.644741380441639
2.2404028498392274 13.875778367305942
3.108920927578234 8.220959613759817
2.392502226913903 12.59231771891625
0.8162402106350586 11.711225480078124
2.204093131744019 10.371915426133864
2.6259721972154098 11.79435494681319
3.50639632901702 9.785969560370276
0.05843791629321766 6.26129303125064
2.7424053755356614 13.026596682216233
3.1111254392713423 14.807662999102869
3.5191760707288897 13.58251809427965
3.8669778095928016 13.746857493475568
3.6822060795387115 12.283491307455643
1.534574157289852 11.240760380448977
0.48752347267736457 10.867333201937457
0.7125851843682073 10.005659868563988
2.2765921314583317 10.361237946599221
2.7049196846404095 11.34385432956885
4.205276688933066 14.3904704

6.572896106537315 19.439786413384674
5.564544916068428 16.71865318628095
5.162440850030969 16.009680590914307
1.8705417244817335 14.400740429897525
2.144481566780637 16.922786958486487
3.58008619889622 17.208317852245756
4.133120945330776 15.868149581102463
1.6082528593686911 16.881075212348467
5.329250807445321 15.22907933748924
4.319861654957127 15.627908289460677
3.60721402388831 13.779923429425564
3.754973402950209 14.521294503310042
4.583303718631521 13.8311106627437
5.306079415245736 14.509332362436727
0.08637310189685636 13.861430331569993
0.6044133457128766 16.117594138350814
0.46704785118065084 16.30683111010851
0.615166359377265 16.6654538114323
2.3050246183551977 15.398814224504331
2.434328259377064 16.893497446663147
1.5129150615593139 12.894657296112996
4.552898058605179 13.811159248972672
4.764071833976628 16.55433830368663
1.3856820104085963 17.863402003688396
1.0667972524215943 15.674714546335101
1.6468470483983315 12.36485481205066
3.260696222316806 15.327366645067439


0.1333355314745069 12.44284820858091
0.22375463563343664 12.972898051313793
0.279055613535785 13.145522333670307
2.1293530217518066 15.014279778571339
1.3993127356166357 14.152148684602803
1.9358738230007486 15.213732929607332
1.8968395373546896 12.406835434078692
1.2070924474621363 6.733484948871096
0.9877691030759229 7.520470291440659
1.2295982876904699 11.90507226773635
3.798316383421479 12.727603180824666
1.7034994523280522 16.271434663200637
2.3110664712416558 15.328399253915482
2.2081270486515794 13.295514972602435
1.0816378771745534 13.665964554073827
1.7973978339866183 15.232748101762757
1.0011521845374274 11.421487555026827
1.935709332370922 17.430187030397967
1.6730744571234029 14.02210680901635
2.9920773758146884 15.608034856822488
2.006980047033382 13.27557416491466
3.1627467797975943 9.43449159393392
2.3330512098745246 16.585049403775955
2.0835840916052373 10.202293059344637
1.6820149328195657 5.394090119944387
2.2276364786435905 9.551719474701521
0.08197231076758009 9.241

1.3648879826822127 14.678848085253984
2.206647250889916 14.220881408552895
2.612161449077736 12.864157854532133
2.074301983069026 16.257996119578213
5.524228571460766 14.686845874122321
6.283667826938131 14.058898236173302
0.11812187652873368 7.790927685756451
0.6403987210795083 8.223724776029938
1.8715180455222582 8.672452162246149
1.4250903853224135 10.090736887125058
2.69176804248163 9.977993694853492
2.449292990329695 9.120566830179069
3.2444972754341275 11.256946229117162
3.1449614168194056 11.072827073987423
2.5587616219254365 13.44146363093611
3.3108265612102543 13.154584254364813
1.543989236539724 14.698507850702137
1.0570619979242313 13.370380379202132
0.4911566455717256 13.985022547135571
0.5853781002353012 14.015347509906992
6.232496347995676 15.609682437509743
5.289085140737518 14.86284496481453
6.523547248925201 14.987960480199346
5.629126288654156 15.318070890053013
5.82854478664019 17.11150572330856
5.373381956579263 18.650108722071632
2.5463478714511134 15.4707680185269

...

3.0566398841557825 18.33980235892361
1.5933289127135202 16.076930121884693
3.3684795629835986 8.52378600866426
2.8302075709829113 8.363749799520784
3.3154141274032187 13.703710975464821
2.4385244735406184 13.436520020494353
2.069025280174165 15.693097492389562
3.2761512587294463 16.33708962605799
4.116559015170842 16.47957186765658
2.4296862314394416 15.101402709321757
2.269497211117856 13.201435877343894
1.475973158590314 14.41823378796112
0.9227433814991726 9.075605942461289
0.07766580603709002 7.500958782698453
0.14912382936377053 11.667710336624074
5.505377089392878 14.492654056027718
2.6809558305596726 14.570429554917096
0.6327799216773751 9.581516265902764
1.1041474275479888 7.914656041368762
3.45689414885857 10.178460190836885
1.7681143946349607 9.72102577073425
1.868622293969708 12.944761734649623
0.8636519783403955 8.227554794231304
1.549067080637817 13.661070780847368
1.3855236654738903 13.480174669993689
1.5558761774875105 13.859731467705508
5.747311508681068 13.902949008567

5.782524207943127 16.547281026160498
3.2333935894150247 16.110657156552975
6.740498814804802 17.886050843467302
4.076090701906283 15.708301679454307
4.14204911862057 16.252841628859123
0.5046398863037652 14.417256218733797
2.4496003255858985 9.92506398184717
2.2959385121727816 11.303441449803183
2.888354897393447 12.16219100734935
1.371154109530937 11.802832968388419
3.961676467867795 14.948151298701376
3.8219235843697157 15.613176524829237
4.626329839544482 14.788470665520988
4.023681415208113 14.526191780193573
4.453997119791684 16.239773092319123
1.478025004566853 15.51747751784445
4.9392069866449635 13.995375732035036
2.0455654420056906 9.196107868504319
1.6660926694897986 12.437594077865807
2.45720656229161 13.843982862658045
3.3685950036452406 15.205286411243048
1.660415564696863 14.59776457597243
3.059132396511058 11.975330252334713
2.0975591007502303 16.29061096189368
3.9178570762885983 14.189423357708822
3.728454624813237 16.495132077188025
3.9680745614201727 16.48115044408571

1.1555565998654442 10.553754842189164
3.5203883181369884 13.735499693941547
3.613182463820832 12.496204707275904
1.94224886535498 12.573341847611482
4.915108878483152 13.969189645381764
4.426467212780744 12.878492456344839
1.3889530217437285 9.403026964029184
0.30441475515167543 7.560392413815084
0.4133506501363739 7.361754854804541
3.13987108912889 10.778101223241025
1.607717976411123 13.591233063809476
3.419782410404144 14.855513486671136
0.5255246776791928 14.780211530506614
5.022140807406139 15.344806791037847
5.415731219152372 16.28320397089507
2.79004634085465 13.009484409333355
2.241211444428074 18.960721642680117
4.16431910216602 17.0123544812934
1.8805173960419541 17.945206036356094
5.59548397892728 14.814224730668434
0.1659526227408177 13.468054778461543
3.156942145498178 17.09438122174067
5.599648660323461 17.35231126005009
1.622275471626749 16.808493971613185
3.2719954888663625 15.689650922817332
6.003222601662511 14.847462381719719
3.881716532897596 14.900969307805106
5.57

2.9312186681542443 16.552991892642762
0.670448262870371 11.454704751715333
0.6175286508551179 13.027563581069977
1.7970334266861907 13.990663254694388
1.653424169370716 14.373276541897896
2.4433388225233656 15.3200202503734
0.47823938443366937 12.23696824425296
0.380445898138391 10.822148371508668
0.6356779381727693 10.254084969371117
0.7865032549344875 11.42806936872615
1.09498082474034 10.116830913884426
2.417160605510965 15.029714215525502
1.273284333230433 14.427178032741162
2.980336721201397 13.932312072648253
0.7345321293253254 7.403943241323474
0.06854739118121934 6.101221988637178
2.266512799875294 7.370808351987738
1.954293157271176 7.868867338565689
3.2955601570423303 9.799866205119532
3.517741878151984 9.516014754858116
0.2156635480443797 8.497836169590308
0.30096922977540863 7.1830184488622395
1.3044996735072156 7.836591034102976
3.988439426669794 9.987940455172666
5.8568025952378395 15.129049122678724
5.363499105971301 14.623193376259831
4.076396610960383 16.05996364536196

0.8255262300930923 8.090281149537624
2.9056473890984273 7.259255750317771
5.3569819775569005 12.914006585943296
1.9123746228114813 16.687148183804617
5.48713430996142 14.260372084282945
1.592209282775309 13.876677516470474
4.61188068593698 13.85011788376191
2.7973191093675105 13.781806485979734
3.4096513947663962 15.817371212610357
1.980047212296492 18.569844624588807
3.381816401683539 19.24325519641502
2.2590572023944038 14.796528459155468
0.9274912518466828 9.0449669095412
1.3506800762643403 15.628315540771386
2.303720156616947 18.076974854730715
1.9168735914323205 16.59415463043561
2.338869277900394 16.347918219732666
2.947827645549446 14.31703733958361
1.0108706147703723 14.493363420130862
4.6320036558653355 14.467446034567454
3.7380811202359006 16.58744991947709
4.856836835266977 19.455294552968386
2.5694396011259575 19.067328778912007
6.763062864408935 16.467997434459974
0.3324083853686044 16.94599116935264
6.3293973051883015 16.49700124971186
4.805194822785291 18.132147458790808

In [102]:
#creating sh_corr_df which will be used for correlations between surface humidity (first column) and all other height levels

#extracting only the spechum column and placing first index in first column, second index in second column etc. for all arrays in spechum

sh_corr_df = sh_df['spechum'].apply(pd.Series)

sh_corr_df.columns = [f'spechum_{i}' for i in range(len(sh_corr_df.columns))]

#filtering such that correlations only go up until 4km (equivalent to :201 columns)
#sh_corr_df = sh_corr_df.iloc[:, :201]

#renaming columns to show #meters above ground
column_mapping = {old_name: str(int(old_name.split('_')[1]) * 20) for old_name in sh_corr_df.columns}

# Rename the columns using the mapping
sh_corr_df = sh_corr_df.rename(columns=column_mapping)

sh_corr_df['near_surface_layer'] = sh_corr_df.apply(lambda row: row[:50].mean(), axis=1)


,0,20,40,60,80,100,120,140,160,180,200,220,240,260,280,300,320,340,360,380,400,420,440,460,480,500,520,540,560,580,600,620,640,660,680,700,720,740,760,780,800,820,840,860,880,900,920,940,960,980,1000,1020,1040,1060,1080,1100,1120,1140,1160,1180,1200,1220,1240,1260,1280,1300,1320,1340,1360,1380,1400,1420,1440,1460,1480,1500,1520,1540,1560,1580,1600,1620,1640,1660,1680,1700,1720,1740,1760,1780,1800,1820,1840,1860,1880,1900,1920,1940,1960,1980,2000,2020,2040,2060,2080,2100,2120,2140,2160,2180,2200,2220,2240,2260,2280,2300,2320,2340,2360,2380,2400,2420,2440,2460,2480,2500,2520,2540,2560,2580,2600,2620,2640,2660,2680,2700,2720,2740,2760,2780,2800,2820,2840,2860,2880,2900,2920,2940,2960,2980,3000,3020,3040,3060,3080,3100,3120,3140,3160,3180,3200,3220,3240,3260,3280,3300,3320,3340,3360,3380,3400,3420,3440,3460,3480,3500,3520,3540,3560,3580,3600,3620,3640,3660,3680,3700,3720,3740,3760,3780,3800,3820,3840,3860,3880,3900,3920,3940,3960,3980,near_surface_layer
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2001-08-01 11:28:00,13.250480,13.345145,12.893328,12.655037,12.746884,13.046898,13.008639,13.404338,13.301301,13.040212,12.768577,12.722844,13.037732,13.129277,12.846598,12.798620,12.927094,12.804274,12.959550,12.788635,12.806487,12.718978,12.753287,12.687906,12.704838,12.788041,12.838494,12.720731,12.885616,12.762519,12.713511,12.493095,12.209470,12.453911,12.519856,12.747228,12.459174,12.653422,12.728642,12.600779,12.506914,12.385143,12.386983,12.357288,12.532772,12.706801,12.779968,12.599423,12.498794,11.929312,11.539701,11.481603,10.975133,10.730225,10.284987,10.416469,10.533027,10.139531,9.969725,9.623714,9.437579,9.541807,9.485752,9.387465,9.267033,9.419633,9.405847,9.279890,9.389892,9.311789,9.469953,9.406142,9.422012,9.163092,11.880759,11.539175,11.406335,10.790407,10.349416,9.854365,9.481289,9.446257,9.341665,9.387308,9.331750,9.237497,9.302344,9.199260,9.299887,9.202620,9.106252,9.188233,9.169871,9.209310,9.257552,9.210904,9.268548,9.228202,9.198603,9.146449,9.205486,9.078163,9.180310,9.129260,9.134031,9.200224,9.121455,9.015930,8.945970,8.869240,9.002050,8.860925,8.932525,8.741971,8.532470,8.209233,8.022438,8.067478,8.033605,8.453375,8.523741,8.418501,7.552167,6.519370,5.463702,4.320583,3.678069,3.344303,3.254300,3.209423,3.168003,3.157411,2.992690,3.117980,3.083233,2.940692,3.027214,3.051899,3.155128,3.162579,3.154960,3.128819,3.136633,3.214942,3.200709,3.183585,3.058446,3.053534,3.285676,3.391567,3.505742,3.513887,4.038085,3.794230,3.465696,3.136873,3.144418,3.142933,2.877779,2.730502,2.715748,2.952679,2.951355,3.028152,2.888901,2.858783,2.885036,3.045658,3.758060,3.826191,3.784954,3.815877,3.966884,4.319299,4.432837,4.569402,4.542474,4.648064,4.598817,4.440770,4.484857,4.334046,4.176580,4.318248,4.384624,4.426232,4.347285,4.262743,4.139584,4.000952,3.986577,3.947512,3.993713,3.979392,3.868904,3.753911,3.691351,3.693772,3.936782,4.194454,12.748097
2001-08-02 11:29:00,13.057998,11.584108,11.630155,11.584313,11.762587,11.921952,11.903122,12.001735,12.110653,12.111579,12.055750,11.942818,12.171288,12.095333,12.013845,12.012866,12.158250,11.996529,12.096320,12.053630,12.072269,12.028715,11.992590,12.111599,12.030415,12.208502,12.106549,12.146224,12.205879,12.075789,12.173977,12.159918,12.073548,11.980451,12.241829,12.198624,12.226920,12.198341,12.218054,12.202263,12.064919,12.255626,12.095159,12.245426,12.090789,12.196186,12.120805,12.252389,12.158545,12.256197,12.186320,12.147374,12.136489,12.035529,11.891586,11.960964,12.020688,11.962395,12.080172,12.182733,12.099465,12.174097,12.104136,12.182458,12.067132,12.128576,12.007357,12.044003,12.016049,12.035154,11.985040,11.963358,12.025469,11.995418,11.997350,11.921147,11.981506,11.873780,12.021016,12.038000,12.034757,12.136676,12.144987,12.019859,11.906180,11.987702,11.847170,11.756120,11.851313,11.876621,11.839456,11.8368

In [104]:
correlations = sh_corr_df.corrwith(sh_corr_df['0'])
column_names = correlations.index  # Collect column names

correlations
# Create a scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(correlations, column_names, marker='o', s=100)  # Swap x and y arguments
plt.xlabel('Correlation Coefficients')
plt.ylabel('Column Names')
plt.title('Correlation Coefficients with Column "0"')

# Show the plot
plt.show()

0                     1.000000
20                    0.999205
40                    0.999160
60                    0.999188
80                    0.999281
100                   0.999273
120                   0.999271
140                   0.999258
160                   0.999215
180                   0.999074
200                   0.998989
220                   0.998528
240                  -0.993250
260                  -0.904496
280                  -0.235171
300                   0.095226
320                   0.097558
340                   0.099014
360                   0.099354
380                   0.098757
400                   0.098388
420                   0.098526
440                   0.097921
460                   0.097430
480                   0.096994
500                   0.096334
520                   0.096331
540                   0.095742
560                   0.095085
580                   0.095127
600                   0.093436
620                   0.092832
640     

In [162]:
y = sh_corr_df['300']
x = sh_corr_df['0']

value_to_find = -1645.2412888167485
index = x.index[x == value_to_find]
index

DatetimeIndex(['2008-06-13 11:31:00'], dtype='datetime64[ns]', name='time', freq=None)

In [165]:
specific_date = pd.to_datetime('2008-06-13 11:31:00')  # Replace with the datetime you want to access

# Use .loc to access the specific row
specific_row = sh_df.loc[specific_date]

In [251]:
specific_row

pres       [12.0 hectopascal, 77.30405521392822 hectopasc...
temp       [3.5 degree_Celsius, 4.7162162363529205 degree...
height     [19.0 meter, 39.0 meter, 59.0 meter, 79.0 mete...
dp         [552.9000244140625 degree_Celsius, 516.7378625...
year                                                    2008
month                                                      6
day                                                       13
hour                                                      11
season                                                   JJA
spechum    [-1645.2412888167485, -1645.5358689579018, -16...
Name: 2008-06-13 11:31:00, dtype: object

In [168]:
sh_df.head()

,pres,temp,height,dp,year,month,day,hour,season,spechum
time,,,,,,,,,,
2001-08-01 11:28:00,"[982.0999755859375 hectopascal, 979.9153911694...","[37.5 degree_Celsius, 36.492308992892504 degre...","[315.0 meter, 335.0 meter, 355.0 meter, 375.0 ...","[18.100000381469727 degree_Celsius, 18.1769234...",2001,8,1,11,JJA,"[13.250480445997486, 13.345145493009868, 12.89..."
2001-08-02 11:29:00,"[984.0 hectopascal, 981.8181596174836 hectopas...","[36.70000076293945 degree_Celsius, 35.89090831...","[315.0 meter, 335.0 meter, 355.0 meter, 375.0 ...","[17.899999618530273 degree_Celsius, 15.9909093...",2001,8,2,11,JJA,"[13.05799786096342, 11.584107749761168, 11.630..."
2001-08-03 11:27:00,"[981.5999755859375 hectopascal, 979.4181962385...","[35.70000076293945 degree_Celsius, 33.36363775...","[315.0 meter, 335.0 meter, 355.0 meter, 375.0 ...","[15.5 degree_Celsius, 12.781818240880966 degre...",2001,8,3,11,JJA,"[11.225772034742164, 9.422988100958097, 9.7172..."
2001-08-04 11:25:00,"[978.9000244140625 hectopascal, 976.7846116274...","[36.5 degree_Celsius, 34.70000076293945 degree...","[315.0 meter, 335.0 meter, 355.0 meter, 375.0 ...","[15.100000381469727 degree_Celsius, 13.0923077...",2001,8,4,11,JJA,"[10.969467586259482, 9.643588657605608, 10.029..."
2001-08-05 11:29:00,"[979.9000244140625 hectopascal, 977.7000121995...","[38.0 degree_Celsius, 35.40000152308494 degree...","[315.0 meter, 335.0 meter, 355.0 meter, 375.0 ...","[16.899999618530273 degree_Celsius, 15.8000001...",2001,8,5,11,JJA,"[12.303832487663191, 11.491106916258312, 11.56..."


In [60]:
directory_path = "../../data/rong1/climate_fu/Data/DOEARM/sgp/sondewnpn"

# file name looks like: sgpsondewnpnC1.b1.20211023.214500.cdf

# all files in directory
files = [
    os.path.join(directory_path, file)
    for file in os.listdir(directory_path)
    if file.endswith(".cdf")
]

filtered_files = []
for file in files:
    # Extract the date part of the file name
    file_name = os.path.basename(file)
    year_month = file_name.split('.')[2]
    month = int(year_month[4:6])
    hours_mins_seconds = file_name.split('.')[3]
    hour = int(hours_mins_seconds[0:2])
    
    if 5 <= month <= 9 and hour == 17:
        print(file_name)
        filtered_files.append(file)

# Open the filtered .cdf files using xr.open_mfdataset
dataset = xr.open_mfdataset(filtered_files)


sgpsondewnpnC1.b1.20160506.172800.cdf
sgpsondewnpnC1.b1.20060824.173200.cdf
sgpsondewnpnC1.b1.20100705.172600.cdf
sgpsondewnpnC1.a1.20020704.172400.cdf
sgpsondewnpnC1.b1.20090620.172800.cdf
sgpsondewnpnC1.b1.20170620.172800.cdf
sgpsondewnpnC1.b1.20100829.172800.cdf
sgpsondewnpnC1.b1.20080827.173600.cdf
sgpsondewnpnC1.b1.20020816.172800.cdf
sgpsondewnpnC1.b1.20170622.172600.cdf
sgpsondewnpnC1.b1.20030824.173000.cdf
sgpsondewnpnC1.b1.20150507.173000.cdf
sgpsondewnpnC1.b1.20100825.172500.cdf
sgpsondewnpnC1.b1.20080917.172700.cdf
sgpsondewnpnC1.b1.20170929.173200.cdf
sgpsondewnpnC1.b1.20160504.172600.cdf
sgpsondewnpnC1.b1.20030917.172800.cdf
sgpsondewnpnC1.b1.20040604.172900.cdf
sgpsondewnpnC1.a1.20020825.172000.cdf
sgpsondewnpnC1.b1.20150611.172600.cdf
sgpsondewnpnC1.b1.20110604.172600.cdf
sgpsondewnpnC1.a1.20020930.172500.cdf
sgpsondewnpnC1.b1.20160903.172900.cdf
sgpsondewnpnC1.b1.20040915.173000.cdf
sgpsondewnpnC1.b1.20110826.172700.cdf
sgpsondewnpnC1.b1.20080819.172800.cdf
sgpsondewnpn

In [61]:
len(filtered_files)

3059